## Imports/Aux

In [1]:
# colab only

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# colab only

%cd /content/gdrive/MyDrive/nlp-final/NLP-Final-Project
! pip3 install -r requirements.txt

/content/gdrive/MyDrive/nlp-final/NLP-Final-Project


In [3]:
%cd /content/gdrive/MyDrive/nlp-final/
%ls
%pwd
#! git clone https://github.com/moses-smt/mosesdecoder.git
%env MOSES=/content/gdrive/MyDrive/nlp-final/mosesdecoder

%env FASTBPE=/content/gdrive/MyDrive/nlp-final/fastBPE
%cd fastBPE
! g++ -std=c++11 -pthread -O3 fastBPE/main.cc -IfastBPE -o fast

/content/gdrive/MyDrive/nlp-final
fairseq/  fastBPE/  mosesdecoder/  NLP-Final-Project/  repo_control.ipynb
env: MOSES=/content/gdrive/MyDrive/nlp-final/mosesdecoder
env: FASTBPE=/content/gdrive/MyDrive/nlp-final/fastBPE
/content/gdrive/MyDrive/nlp-final/fastBPE


In [12]:
# I'm trying to make new dict.txt/bpecodes for our data (with brain regions filled in)
# to solve the problem I was getting, which is that when I try to train the model
# on the filled-in data, it struggles to understand the language.


%cd /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/tagged_data
!perl ${MOSES}/scripts/tokenizer/tokenizer.perl -l en -a -threads 8 < /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/tagged_data/train_tags.x > /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/tagged_data/train_tags.tok.x
!perl ${MOSES}/scripts/tokenizer/tokenizer.perl -l en -a -threads 8 < /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/tagged_data/train_tags.y > /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/tagged_data/train_tags.tok.y

/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/tagged_data
Tokenizer Version 1.1
Language: en
Number of threads: 8
Tokenizer Version 1.1
Language: en
Number of threads: 8


In [13]:
%cd /content/gdrive/MyDrive/nlp-final/fastBPE
! ./fast getvocab /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/tagged_data/train_tags.tok.x /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/tagged_data/train_tags.tok.y > /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/tagged_data/tagged_vocab.txt

/content/gdrive/MyDrive/nlp-final/fastBPE
Loading vocabulary from /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/tagged_data/train_tags.tok.x ...
Read 57418 words (6075 unique) from text file.
Loading vocabulary from /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/tagged_data/train_tags.tok.y ...
Read 22699 words (6083 unique) from text file.


In [14]:
import string
%cd /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/tagged_data

with open('tagged_vocab.txt') as f1:
  tagged_lines = f1.readlines()

with open('dict.txt') as f2:
  dict_lines = f2.readlines()

d2 = {}
for line in tagged_lines:
  #line = line.translate(str.maketrans('', '', string.punctuation))
  split_line = line.split(' ')
  d2[split_line[0]] = int(split_line[1])

d1 = {}
for line in dict_lines:
  split_line = line.split(' ')
  d1[split_line[0]] = int(split_line[1])

for k, v in d2.items():
  k1 = k
  if k in d1:
    d1[k] += int(v)
  else:
    d1[k] = int(v)

with open('comb_dict.txt', 'w') as f3:
  for k, v in d1.items():
    f3.write(k + " " + str(v) + "\n")

/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/tagged_data


In [15]:
# after the preceeding cell, I manually deleted vocab words that aren't useful
# for our purposes so it matched the BioGPT vocab length

# if this fails, could see how to replicate the current bpecodes

%cd /content/gdrive/MyDrive/nlp-final/fastBPE

! ./fast learnbpe 40000 /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/tagged_data/train_tags.tok.x /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/tagged_data/train_tags.tok.y > /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/tagged_data/my_bpecodes

# using these bpe codes failed, but also possibly because I generated them
# with the blanks data and tried to use them on the pos-only data

/content/gdrive/MyDrive/nlp-final/fastBPE
Loading vocabulary from /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/tagged_data/train_tags.tok.x ...
Read 57418 words (6075 unique) from text file.
Loading vocabulary from /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/tagged_data/train_tags.tok.y ...
Read 22699 words (6083 unique) from text file.


In [4]:
%cd /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/utils

! bash preprocess.sh

/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/utils
Preprocessing train
Tokenizer Version 1.1
Language: en
Number of threads: 8
Tokenizer Version 1.1
Language: en
Number of threads: 8
Loading codes from ../data/tagged_data/pos_only/bpecodes ...
Read 40000 codes from the codes file.
Loading vocabulary from ../data/tagged_data/pos_only/relis_train.tok.x ...
Read 12194 words (2246 unique) from text file.
Applying BPE to ../data/tagged_data/pos_only/relis_train.tok.x ...
Modified 12194 words from text file.
Loading codes from ../data/tagged_data/pos_only/bpecodes ...
Read 40000 codes from the codes file.
Loading vocabulary from ../data/tagged_data/pos_only/relis_train.tok.y ...
Read 5222 words (539 unique) from text file.
Applying BPE to ../data/tagged_data/pos_only/relis_train.tok.y ...
Modified 5222 words from text file.
Preprocessing valid
Tokenizer Version 1.1
Language: en
Number of threads: 8
Tokenizer Version 1.1
Language: en
Number of threads: 8
Loading codes from ../data/

In [5]:
%cd /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/utils
! bash train.sh

/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/utils
2023-04-23 23:56:21.403440: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 23:56:22.251530: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-23 23:56:23 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2023-04-23 23:56:23 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-04-23 23:56:28 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu':

In [5]:
# colab only

import sys
sys.path.append('/content/drive/MyDrive/NLP_Final_Project/src')

In [6]:
%cd /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src

data_dir = '/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/relis-bin'
model_dir = '/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/checkpoints/RE-WhiteText-BioGPT'
model_file = 'checkpoint_avg_masked_pos.pt'
src_file = '/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/data/blanks/relis_valid.tok.bpe.x'
output_file = 'pos_valid_inference_18_epochs.txt'
beam = 1
decoding_length = 1024

#! python utils/average_checkpoints.py --inputs='/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/checkpoints/blanks/RE-WhiteText-BioGPT' --output='/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src/checkpoints/RE-WhiteText-BioGPT/blanks/checkpoint_avg.pt' --num-epoch-checkpoints=2

/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src


## Model Setup

In [7]:
%cd /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src

from src.transformer_lm_prompt import TransformerLanguageModelPrompt

src_inputs = []
with open(src_file) as reader:
    for line in reader:
        src_inputs.append(line.strip())
    

m = TransformerLanguageModelPrompt.from_pretrained(
    model_dir, 
    model_file, 
    data_dir,
    max_len_b=decoding_length,
    max_tokens=12000,)

if m.cfg.common.fp16:
    print('Converting to float 16')
    m.half()
m.cuda()

outputs = m.sample(src_inputs, beam=beam)

with open(f"{output_file}", "w", encoding='utf8') as fw:
    for i in range(len(outputs)):
        fw.write(outputs[i] + '\n')

/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src


In [8]:
# debpe
! sed -i "s/@@ //g" pos_valid_inference_18_epochs.txt
# detok
! perl ${MOSES}/scripts/tokenizer/detokenizer.perl -l en -a < pos_valid_inference_18_epochs.txt > pos_valid_inference_18_epochs.txt.detok

Detokenizer Version $Revision: 4134 $
Language: en


## Calculating metrics

In [10]:
%cd /content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src

with open('pos_valid_inference_18_epochs.txt.detok') as f1:
  pred_lines = f1.readlines()

with open('data/blanks/relis_valid.y') as f2:
  gold_lines = f2.readlines()

true_pos = 0
false_pos = 0
false_neg = 0

template = "the relation between BR1 and BR2 exists.\n".strip()

for pred, gold in zip(pred_lines, gold_lines):
  split_line = pred.split("learned9")
  gold = gold.strip()
  
  if gold == template:
    if split_line[1].strip() == template:
      true_pos += 1
    else:
      false_neg += 1
  else:
    if split_line[1].strip() == template:
      false_pos += 1

precision = true_pos/(true_pos + false_pos)
recall = true_pos/(true_pos + false_neg)

print("Precision = ", precision)
print("Recall = ", recall)
print("F1 = ", 2*((precision*recall)/(precision + recall)))
print("F2 = ", 5*((precision*recall)/((4*precision) + recall)))

/content/gdrive/MyDrive/nlp-final/NLP-Final-Project/src
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
